# STOP!!! READ THIS FIRST
You should only look at these solutions once you think you have working solutions of your own :).

## Section 4 - Uncertainties in best-fit parameters
This section details how you can estimate uncertainties on your best fit parameters. For the line example we have the slope $b$ and the intercept $m$, and we want to know $\sigma_b$ and $\sigma_m$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc("text", usetex=True, fontsize=24) #If this gives a warning just ignore it
plt.rc("errorbar", capsize=3) #Ignore

/home/tom/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
